In [ ]:
# Reference code from Session 13 - Heaps and priority queues

import heapq
import math

class Task:
    """
    - id: Task id (a reference number)   
    - description: Task short description   
    - duration: Task duration in minutes   
    - dependencies: List of task ids that need to preceed this task
    - earliest_start_time: The earliest time (in minutes) the task can start
    - latest_start_time: The latest time (in minutes) the task must start
    - status: Current status of the task     
    """
    
    # Initializes an instance of Task
    def __init__(self, id, description, duration,
                 dependencies, earliest_start_time=None, latest_start_time=None, status="N"):
        self.id = id
        self.description = description
        self.duration = duration
        self.dependencies = dependencies
        self.earliest_start_time = earliest_start_time
        self.latest_start_time = latest_start_time
        self.status = status

    def __lt__(self, other):
        # This priority logic IS correct and stays the same.
        # It ensures fixed tasks (earliest_start_time) are
        # always at the front of the queue.
        
        # 1. Get priority for earliest_start_time (None is infinity, so lowest prio)
        prio_self_e = self.earliest_start_time if self.earliest_start_time is not None else math.inf
        prio_other_e = other.earliest_start_time if other.earliest_start_time is not None else math.inf
        
        # 2. Get priority for latest_start_time (None is infinity, so lowest prio)
        prio_self_l = self.latest_start_time if self.latest_start_time is not None else math.inf
        prio_other_l = other.latest_start_time if other.latest_start_time is not None else math.inf

        # 3. Create priority tuples
        self_tuple = (prio_self_e, prio_self_l, self.id)
        other_tuple = (prio_other_e, prio_other_l, other.id)
        
        return self_tuple < other_tuple
    
class TaskScheduler:
    """
    An Advanced Daily Task Scheduler Using a Multi-Priority Queue
    """
    NOT_STARTED = 'N'
    IN_PRIORITY_QUEUE = 'I'
    COMPLETED = 'C'
    
    def __init__(self, tasks):
        self.tasks = tasks
        self.task_map = {task.id: task for task in tasks} 
        self.priority_queue = []
        self.time_blocked_tasks = []
        
    def print_self(self):
        print("Tasks added to the advanced scheduler:")
        print("---------------------------------------")
        for t in self.tasks:
            print(f"➡️'{t.description}', duration = {t.duration} mins.")   
            
            if t.dependencies:
                try:
                    dep_descriptions = [self.task_map[dep_id].description for dep_id in t.dependencies]
                    print(f"\t ⚠️ Depends on: \"{', '.join(dep_descriptions)}\"")
                except KeyError as e:
                    print(f"\t ⚠️ Error: Dependency ID {e} not found!")
            
            if t.earliest_start_time and t.latest_start_time:
                if t.earliest_start_time == t.latest_start_time:
                    print(f"\t ⏰ FIXED: Must start exactly at {self.format_time(t.earliest_start_time)}")
                else:
                    print(f"\t ⏰ WINDOW: Must run between {self.format_time(t.earliest_start_time)} and {self.format_time(t.latest_start_time)}")
            elif t.earliest_start_time:
                print(f"\t ⏰ EARLIEST: Cannot start before {self.format_time(t.earliest_start_time)}")
            elif t.latest_start_time:
                print(f"\t ⏰ DEADLINE: Must start by {self.format_time(t.latest_start_time)}")
            else:
                print(f"\t ⏰ FLEXIBLE: No time constraints")
            
    def remove_dependency(self, id):
        for t in self.tasks:
            if t.id != id and id in t.dependencies:
                t.dependencies.remove(id)           
            
    def get_tasks_ready(self):
        """Finds tasks with no dependencies and moves them to the queue."""
        for task in self.tasks:
            if task.status == self.NOT_STARTED and not task.dependencies: 
                task.status = self.IN_PRIORITY_QUEUE 
                heapq.heappush(self.priority_queue, task)
    
    def move_ready_time_blocked_tasks(self, current_time):
        """
        Moves tasks from the time-blocked list back to the priority queue
        if their start time has been reached.
        """
        tasks_to_move = []
        for task in self.time_blocked_tasks:
            # Check if task is not None and its start time is met
            if task and (task.earliest_start_time is None or current_time >= task.earliest_start_time):
                tasks_to_move.append(task)
        
        for task in tasks_to_move:
            self.time_blocked_tasks.remove(task)
            heapq.heappush(self.priority_queue, task)

    def check_unscheduled_tasks(self):
        """Checks if any tasks are still not completed."""
        for task in self.tasks:
            if task.status != self.COMPLETED:
                return True
        return False   
    
    def format_time(self, time):
        if not isinstance(time, (int, float)):
            return "Invalid time"
        days = int(time) // 1440
        minutes_today = int(time) % 1440
        hour = minutes_today // 60
        minute = minutes_today % 60
        if days > 0:
            return f"Day {days+1}, {hour}h{minute:02d}"
        return f"{hour}h{minute:02d}"
    
    def run_task_scheduler(self, starting_time):
        current_time = starting_time
        print("Running an advanced scheduler:\n")
        
        while self.check_unscheduled_tasks():
            
            self.get_tasks_ready()
            self.move_ready_time_blocked_tasks(current_time)
            
            if self.priority_queue:
                task = heapq.heappop(self.priority_queue)
                
                # Check 1: Is it too early to run this task?
                if task.earliest_start_time is not None and current_time < task.earliest_start_time:
                    self.time_blocked_tasks.append(task)
                    continue
                
                # --- NEW LOGIC: THE "LOOK-AHEAD" FIX ---
                # Before running this task, check if it will conflict
                # with a future, higher-priority task.
                
                # Find the earliest start time of any task *waiting* in the time_blocked_list
                next_hard_start_time = math.inf
                if self.time_blocked_tasks:
                    valid_times = [t.earliest_start_time for t in self.time_blocked_tasks if t.earliest_start_time is not None]
                    if valid_times:
                        next_hard_start_time = min(valid_times)
                
                task_end_time = current_time + task.duration
                
                # This is the conflict check you wanted:
                if task_end_time > next_hard_start_time:
                    # CONFLICT! This flexible task would make us late.
                    print(f"🕰t={self.format_time(current_time)}... Delaying flexible task '{task.description}' to keep {self.format_time(next_hard_start_time)} free.")
                    
                    # Put this task back on the queue to be run later
                    heapq.heappush(self.priority_queue, task)
                    
                    # We must now "wait" until the fixed task.
                    current_time = next_hard_start_time
                    continue # Restart the loop
                # --- END OF NEW LOGIC ---

                
                # If we pass both checks, we can run the task.
                
                # Check for deadline violation (running LATE)
                if task.latest_start_time is not None and current_time > task.latest_start_time:
                    print(f"🕰t={self.format_time(current_time)} - ⚠️ WARNING: Running '{task.description}' LATE (Deadline was {self.format_time(task.latest_start_time)})")
                
                # --- Execute Task ---
                print(f"🕰t={self.format_time(current_time)}")
                print(f"\tstarted '{task.description}' for {task.duration} mins...")
                current_time += task.duration            
                print(f"\t✅ t={self.format_time(current_time)}, task completed!") 
                
                self.remove_dependency(task.id)
                task.status = self.COMPLETED
            
            elif self.time_blocked_tasks:
                # No tasks are runnable *right now*, but we are waiting for some.
                # Advance time to the next available task.
                try:
                    next_available_time = min(t.earliest_start_time for t in self.time_blocked_tasks if t.earliest_start_time is not None)
                except ValueError:
                     print(f"🕰t={self.format_time(current_time)} - ERROR: Time-blocked, but no valid start time!")
                     break
                
                if current_time < next_available_time:
                    print(f"🕰t={self.format_time(current_time)}... no tasks runnable. Waiting until {self.format_time(next_available_time)}...")
                    current_time = next_available_time
            
            elif self.check_unscheduled_tasks():
                 # Deadlock
                 print(f"🕰t={self.format_time(current_time)} - ERROR: Deadlock detected!")
                 # ... (rest of deadlock logic) ...
                 break
            
        total_time = current_time - starting_time             
        print(f"\n🏁 Completed all planned tasks in {int(total_time)//60}h{int(total_time)%60:02d}min!")

In [54]:
# Reference code from Session 13 - Heaps and priority queues

"""
List of Tasks - 5 Nov 2025:

• Fixed tasks:
1) Take CS111 class

• Partly Timed Tasks
1) Wake up at 09:30
2) Have lunch at 14:30
3) Buy Bento at Lincos after gym after 20:00
4) Go to bed before 02:30

• Non-timely tasks:
1) Make CS113 Pre-Class Work
2) Have Japan's signature drink - Matcha Green Tea
3) Go to the gym
4) Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos
5) Take shower after gym
6) Have dinner

"""
tasks = [
    Task(id=0, description='Wake up at 09:30', 
         duration=10, dependencies=[], earliest_start_time=570), 
    Task(id=1, description='Have lunch at 14:30', 
         duration=60, dependencies=[0], earliest_start_time=870), 
    Task(id=2, description='Go to the gym', 
         duration=120, dependencies=[0, 1]), 
    Task(id=3, description='Buy Bento at Lincos after gym after 20:00', 
         duration=25, dependencies=[0, 2], earliest_start_time=1200), 
    Task(id=4, description='Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos', 
         duration=10, dependencies=[0, 1, 3]), 
    Task(id=5, description='Take shower after gym', 
         duration=30, dependencies=[0, 2]), 
    Task(id=6, description='Have dinner', 
         duration=40, dependencies=[0, 5]), 
    Task(id=7, description='Prepare luggage for Kamaishi Trip', 
         duration=100, dependencies=[0, 6]),
     Task(id=8, description='Make CS113 Pre-Class Work', 
         duration=120, dependencies=[0, 7]),
     Task(id=9, description="Have Japan's signature drink - Matcha Green Tea", 
         duration=15, dependencies=[0]),
     Task(id=10, description='Go to bed before 02:30', 
         duration=15, dependencies=[0, 1, 6, 7, 8]),
     Task(id=11, description="Attend CS111 Session from 16:00 to 17:30",
          duration=90, dependencies=[0], earliest_start_time=16*60, latest_start_time=17*60+30)
    ]

task_scheduler = TaskScheduler(tasks)

# print the scheduler's input 
task_scheduler.print_self()

Tasks added to the advanced scheduler:
---------------------------------------
➡️'Wake up at 09:30', duration = 10 mins.
	 ⏰ EARLIEST: Cannot start before 9h30
➡️'Have lunch at 14:30', duration = 60 mins.
	 ⚠️ Depends on: "Wake up at 09:30"
	 ⏰ EARLIEST: Cannot start before 14h30
➡️'Go to the gym', duration = 120 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Have lunch at 14:30"
	 ⏰ FLEXIBLE: No time constraints
➡️'Buy Bento at Lincos after gym after 20:00', duration = 25 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Go to the gym"
	 ⏰ EARLIEST: Cannot start before 20h00
➡️'Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos', duration = 10 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Have lunch at 14:30, Buy Bento at Lincos after gym after 20:00"
	 ⏰ FLEXIBLE: No time constraints
➡️'Take shower after gym', duration = 30 mins.
	 ⚠️ Depends on: "Wake up at 09:30, Go to the gym"
	 ⏰ FLEXIBLE: No time constraints
➡️'Have dinner', duration = 40 mins.
	 ⚠️ Depends on: "Wake up at 09:3

In [55]:
# Reference code from Session 13 - Heaps and priority queues
start_scheduler_at = int((9.5)*60)
task_scheduler.run_task_scheduler(start_scheduler_at)

Running an advanced scheduler:

🕰t=9h30
	started 'Wake up at 09:30' for 10 mins...
	✅ t=9h40, task completed!
🕰t=9h40
	started 'Have Japan's signature drink - Matcha Green Tea' for 15 mins...
	✅ t=9h55, task completed!
🕰t=9h55... no tasks runnable. Waiting until 14h30...
🕰t=14h30
	started 'Have lunch at 14:30' for 60 mins...
	✅ t=15h30, task completed!
🕰t=15h30... Delaying flexible task 'Go to the gym' to keep 16h00 free.
🕰t=16h00
	started 'Attend CS111 Session from 16:00 to 17:30' for 90 mins...
	✅ t=17h30, task completed!
🕰t=17h30
	started 'Go to the gym' for 120 mins...
	✅ t=19h30, task completed!
🕰t=19h30
	started 'Take shower after gym' for 30 mins...
	✅ t=20h00, task completed!
🕰t=20h00
	started 'Buy Bento at Lincos after gym after 20:00' for 25 mins...
	✅ t=20h25, task completed!
🕰t=20h25
	started 'Buy 100¥ (~$0.65) Snacks for Kamaishi Trip at Mini Stop after Lincos' for 10 mins...
	✅ t=20h35, task completed!
🕰t=20h35
	started 'Have dinner' for 40 mins...
	✅ t=21h15, task comple